In [10]:
import pandas as pd

In [11]:
df = pd.read_json("../다산콜센터/민원(콜센터) 질의응답_다산콜센터_일반행정 문의_Training.json")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50336 entries, 0 to 50335
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   도메인        50336 non-null  object
 1   카테고리       50336 non-null  object
 2   대화셋일련번호    50336 non-null  object
 3   화자         50336 non-null  object
 4   문장번호       50336 non-null  int64 
 5   고객의도       50336 non-null  object
 6   상담사의도      50336 non-null  object
 7   QA         50336 non-null  object
 8   고객질문(요청)   50336 non-null  object
 9   상담사질문(요청)  50336 non-null  object
 10  고객답변       50336 non-null  object
 11  상담사답변      50336 non-null  object
 12  개체명        50336 non-null  object
 13  용어사전       50336 non-null  object
 14  지식베이스      50336 non-null  object
dtypes: int64(1), object(14)
memory usage: 5.8+ MB


In [13]:
# 결측치 개수 확인
df.isna().sum()
# 결측치 없음

도메인          0
카테고리         0
대화셋일련번호      0
화자           0
문장번호         0
고객의도         0
상담사의도        0
QA           0
고객질문(요청)     0
상담사질문(요청)    0
고객답변         0
상담사답변        0
개체명          0
용어사전         0
지식베이스        0
dtype: int64

In [14]:
df.head()
# 결측치 있음
# -> 상세한 답변을 위해, 답변 대신 추가로 질문하는 경우가 있으므로

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,일반행정 문의,B2240,고객,1,지방세납부,,Q,지방세를 내려면 어떻게 해야됩니까?,,,,지방세,지방세/세금,"지방세,세금"
1,다산콜센터,일반행정 문의,B2240,상담사,2,,지방세납부,A,,,,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,"은행, 사이트, 지방세, 납부",은행/공공기관/ 지방세/세금,"사이트,세금"
2,다산콜센터,일반행정 문의,B2240,고객,3,지방세납부,,Q,은행 어플에서도 됩니까?,,,,"은행, 어플",은행/공공기관,"어플,공공기관"
3,다산콜센터,일반행정 문의,B2240,상담사,4,,지방세납부,Q,,어떤 은행을 이용하고 계십니까?,,,은행,은행/공공기관,"은행,공공기관"
4,다산콜센터,일반행정 문의,B2240,고객,5,지방세납부,,A,,,기업은행을 이용하고 있습니다.,,기업은행,기업은행/상호,"기업은행,상호"


---
#### 문제
1. 일반행정 데이터와 대중교통 데이터 로드
2. 두 개의 데이터프레임을 결합 (단순 행 결합)
3. 데이터 필터링 - 고객 질문에 대한 상담사 답변이 즉각적으로 오는 데이터들만 필터링
4. 중복되는 질문 데이터 제거
5. 질문들을 모아서 토큰화, 벡터화
6. 그 외의 질문 목록을 이용하여 코사인 유사도 확인 & 유사 질문과 답변 출력

In [69]:
# 질문 목록
new_questions = [
    '여권 재발급 신청 방법을 알려줘',
    '전입 신고가 인터넷으로 가능한가요?',
    '지방세 환급금을 어디서 신청하나요?'
]

In [70]:
# 2개의 데이터프레임 로드
df1 = pd.read_json("../다산콜센터/민원(콜센터) 질의응답_다산콜센터_일반행정 문의_Training.json")
df2 = pd.read_json("../다산콜센터/민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.json")

In [76]:
# 단순 행 결합 (union 결합)
# 주의) 로드 시 인덱스를 따로 지정하지 않았기 때문에, 단순 결합 시 인덱스가 중첩됨
df_combined = pd.concat([df1, df2], axis=0)

In [74]:
df_combined.loc[0, ]

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,일반행정 문의,B2240,고객,1,지방세납부,,Q,지방세를 내려면 어떻게 해야됩니까?,,,,지방세,지방세/세금,"지방세,세금"
0,다산콜센터,대중교통 안내,B2033,고객,1,버스노선,,Q,서울 가산동에서 남대문시장가는 버스노선을 알고싶습니다,,,,"서울, 가산동, 남대문시장, 버스, 노선",서울/지명/ 가산동/동네/ 남대문시장/지명/ 버스/교통수단,"가산동,교통수단"


In [78]:
# 따라서 Primary Key(중복 X, null X)를 유지하기 위해 ignore_index=True 필수
df_combined = pd.concat([df1, df2], axis=0, ignore_index=True)
df_combined.head()

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,다산콜센터,일반행정 문의,B2240,고객,1,지방세납부,,Q,지방세를 내려면 어떻게 해야됩니까?,,,,지방세,지방세/세금,"지방세,세금"
1,다산콜센터,일반행정 문의,B2240,상담사,2,,지방세납부,A,,,,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.,"은행, 사이트, 지방세, 납부",은행/공공기관/ 지방세/세금,"사이트,세금"
2,다산콜센터,일반행정 문의,B2240,고객,3,지방세납부,,Q,은행 어플에서도 됩니까?,,,,"은행, 어플",은행/공공기관,"어플,공공기관"
3,다산콜센터,일반행정 문의,B2240,상담사,4,,지방세납부,Q,,어떤 은행을 이용하고 계십니까?,,,은행,은행/공공기관,"은행,공공기관"
4,다산콜센터,일반행정 문의,B2240,고객,5,지방세납부,,A,,,기업은행을 이용하고 있습니다.,,기업은행,기업은행/상호,"기업은행,상호"


In [ ]:
# 고객질문(요청) 컬럼의 데이터 개수 확인
df_combined['고객질문(요청)'].value_counts()
# 여러 개의 공백 데이터가 존재함을 확인함 (공백1 58725개, 공백2 78개, ...)

고객질문(요청)
                             58725
카드결제와 현금결제할 때 요금 차이가 있나요?       79
                                78
버스요금은 얼마인가요?                    55
시간은 얼마나 걸려요?                    54
                             ...  
금산 시내버스 첫차가 몇 시 인가요?             1
그럼 금산 시내버스 막차는 몇 시 인가요?          1
혹시 영동에 지하철 있나요?                  1
가평 시내버스 첫차가 몇 시 인가요?             1
그럼 가평 시내버스 막차는 몇 시 인가요?          1
Name: count, Length: 22824, dtype: int64

In [ ]:
# 공백으로 이루어진 value들을 하나로 통일
# Series에서 각각의 value를 추출하여 함수에 대입 - map()

# df_combined['고객질문(요청)'].map( lambda x: x.strip() ).value_counts()
# 모든 데이터프레임의 공백값을 바꿔야 함

# df_combined.map( lambda x: x.strip() ).value_counts()
# strip()은 문자열 함수인데, 전체 데이터프레임 중엔 int 타입 데이터도 있으므로 Error

# 문자로 바꿔줌
df_combined.map( lambda x: str(x).strip() ).value_counts()

고객질문(요청)
                             58807
카드결제와 현금결제할 때 요금 차이가 있나요?       79
버스요금은 얼마인가요?                    55
시간은 얼마나 걸려요?                    55
안녕하세요.                          46
                             ...  
익산 시내버스 첫차가 몇 시 인가요?             1
그럼 익산 시내버스 막차는 몇 시 인가요?          1
순천 시내버스 첫차가 몇 시 인가요?             1
논산 시내버스 첫차가 몇 시 인가요?             1
참석했다가 코로나 걸리면 보상해줍니까?            1
Name: count, Length: 22721, dtype: int64

In [ ]:
# 3. 데이터 필터링 - '즉각적인 답변' 추출
# 주의) 필터링 전 중복 질문 먼저 제거하면 답변도 다 사라짐

# Shift 함수를 사용하여 다음 행의 데이터를 현재 행으로 가져옴
df_combined['next_화자'] = df_combined['화자'].shift(-1)
df_combined['next_문장번호'] = df_combined['문장번호'].shift(-1)
df_combined['next_QA'] = df_combined['QA'].shift(-1)
df_combined['next_상담사답변'] = df_combined['상담사답변'].shift(-1)
df_combined['next_대화셋'] = df_combined['대화셋일련번호'].shift(-1)


# 조건 설정 (Gemini 답변):
# 1. 현재 행은 고객 질문 (화자='고객', QA='Q')
condition_customer_q = (df['화자'] == '고객') & (df['QA'] == 'Q')

# 2. 다음 행은 상담사 답변 (next_화자='상담사', next_QA='A')
condition_counselor_a = (df['next_화자'] == '상담사') & (df['next_QA'] == 'A')

# 3. 다음 행은 바로 다음 문장이어야 함 (문장번호 연속성)
condition_continuous = df['next_문장번호'] == (df['문장번호'] + 1)

# 4. 같은 대화셋일련번호 내에서 발생해야 함
condition_same_dialog = df['대화셋일련번호'] == df['next_대화셋']

# 모든 조건을 만족하는 행만 필터링
df_qa = df[
    condition_customer_q & 
    condition_counselor_a & 
    condition_continuous & 
    condition_same_dialog
].copy()

# 필요한 컬럼만 추출하여 최종 QA 데이터셋 구성
df_final = pd.DataFrame({
    '대화셋일련번호': df_qa['대화셋일련번호'],
    '질문': df_qa['고객질문(요청)'],
    '답변': df_qa['next_상담사답변']
}).dropna().reset_index(drop=True) # 질문이나 답변에 NaN이 남아있을 경우 제거

df_final

,대화셋일련번호,질문,답변
0,B2240,지방세를 내려면 어떻게 해야됩니까?,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.
1,B2240,은행을 직접방문해도 됩니까?,방문납부도 가능합니다.
2,B2240,버스로 가는 방법도 있습니까?,도보로 가시는게 빠를 것 같습니다.
3,B2240,다른 납부방법도 있습니까?,위택스 사이트에서 납부하실수 있습니다.
4,B2240,사이트 주소가 어떻게 됩니까?,www.wetax.go.kr 입니다.
...,...,...,...
15783,B35809,임대료는 어떻게 되나요?,62400원 입니다.
15784,B35809,보증금도 있나요?,1423200원 입니다.
15785,B35809,신청은 어떻게 하나요?,"입주신청서와 추천서, 급여내역서 외 기타 서류를 갖고 홈페이지에서 신청하면 됩니다."
15786,B35809,입주순위도 있나요?,1~3순위가 있습니다.


In [41]:
# 4. 중복되는 질문 데이터 제거
# '질문' 컬럼을 기준으로 중복되는 행을 제거 (가장 먼저 나온 질문 유지)
df_final.drop_duplicates(subset=['질문'], keep='first', inplace=True)
df_final

,대화셋일련번호,질문,답변
0,B2240,지방세를 내려면 어떻게 해야됩니까?,이용하시는 은행의 사이트에서 지방세 납부가 가능합니다.
1,B2240,은행을 직접방문해도 됩니까?,방문납부도 가능합니다.
2,B2240,버스로 가는 방법도 있습니까?,도보로 가시는게 빠를 것 같습니다.
3,B2240,다른 납부방법도 있습니까?,위택스 사이트에서 납부하실수 있습니다.
4,B2240,사이트 주소가 어떻게 됩니까?,www.wetax.go.kr 입니다.
...,...,...,...
15782,B35809,꼭 서울시에 근무해야하나요?,서울소재 직장근무로 제한하고있습니다.
15783,B35809,임대료는 어떻게 되나요?,62400원 입니다.
15784,B35809,보증금도 있나요?,1423200원 입니다.
15786,B35809,입주순위도 있나요?,1~3순위가 있습니다.


In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import re

In [43]:
# 5. 질문들을 모아서 토큰화, 벡터화 (TF-IDF 사용)
# 토큰화 및 벡터화를 위한 질문 목록 준비
questions = df_final['질문'].tolist()
answers = df_final['답변'].tolist()

In [44]:
# Okt 초기화
okt = Okt()

# 토큰화 함수 정의
def tokenize(text):
    # 특수문자 제거 및 정규화 추가
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    return okt.morphs(text)

# 벡터화
tfidf = TfidfVectorizer(
    tokenizer= tokenize,
    ngram_range= (1, 2),
    min_df= 1
)

In [45]:
question_vec = tfidf.fit_transform(questions)
question_vec

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 162623 stored elements and shape (12439, 40362)>

In [ ]:
# 6. 코사인 유사도 확인 및 유사 질문/답변 출력
# 함수 정의
def find_similar_qa(queries, top_k=3):
    # 질문들(Queries) 토큰화 및 벡터화 (fit이 아닌 transform 사용)
    query_vector = tfidf.transform(queries)
    
    # 코사인 유사도 계산
    # (쿼리 벡터와 모든 질문 벡터 간의 유사도)
    sim_scores = cosine_similarity(query_vector, question_vec)
    
    # 각 query 별 결과 출력
    for q_idx, query in enumerate(queries):

        # 현재 쿼리에 해당하는 유사도 점수 배열 가져오기
        sims = sim_scores[q_idx].ravel()

        # 유사도가 높은 인덱스 k개 찾기
        # argsort로 정렬 후, 뒤에서부터 top_k개를 선택
        top_indices = sims.argsort()[::-1][:top_k]
    
        print(f"\n[유사 질문 분석 결과: '{query}']")
        print("=" * 40)
    
        for i in top_indices:
            # i는 기존 질문 목록(questions, answers)의 인덱스
            # 자기 자신을 찾을 경우 건너뛸 수 있도록 추가 필터링 로직 필요하지만, 여기서는 중복 제거했으므로 생략
            score = sims[i]
            print(f"유사 질문(유사도): {questions[i]} ({score * 100 :.2f}%)")
            print(f"- 답변: {answers[i]}")
            
        print("\n" + "="*50 + "\n") # 각 쿼리 결과 구분선

In [57]:
find_similar_qa(new_questions)


[유사 질문 분석 결과: '여권 재발급 신청 방법을 알려줘']
유사 질문(유사도): 여권 재발급 비용도 알 수있나요? (41.15%)
 답변: 복수여권53000원,일반여권 50000원입니다.
유사 질문(유사도): 신청방법을 알려주세요. (40.45%)
 답변: 주민등록상 세대주와 가까운 주민센터 또는 복지로 홈페이지에서 신청가능하세요.
유사 질문(유사도): 재발급 신청은 어떻게 해야돼요? (39.91%)
 답변: 인터넷, 방문신청 둘다 가능합니다.



[유사 질문 분석 결과: '전입 신고가 인터넷으로 가능한가요?']
유사 질문(유사도): 인터넷으로 가능한가요? (61.90%)
 답변: 인터넷으로 신청 가능합니다.
유사 질문(유사도): 납부는 인터넷으로 가능한가요? (43.65%)
 답변: 네. 가능합니다.
유사 질문(유사도): 전입신고 비용은 얼마에요? (28.00%)
 답변: 전입신고 수수료는 무료입니다



[유사 질문 분석 결과: '지방세 환급금을 어디서 신청하나요?']
유사 질문(유사도): 어디서 신청하나요? (54.36%)
 답변: 온라인청년센터 통해서 신청가능합니다!
유사 질문(유사도): 지방세 환급금 신청은 어떻게 해야하죠? (41.77%)
 답변: 인터넷에서 접수를 하셔야 합니다
유사 질문(유사도): 신청을 어디서 하나요? (37.01%)
 답변: 주민등록상 거주지가 속한 시, 군또는 자기추내의 읍, 면, 동 주민센터에서 발급 가능합니다. 




---
#### 문제 2
고객 질문의 데이터를 이용하여 카테고리를 분류하는 모델을 생성
- 고객 질문 데이터들을 이용하여 토큰화, 벡터화 작업 (독립 변수)
- 타겟 데이터는 '일반행정', '대중교통' 카테고리 (종속 변수)
    - 카테고리 데이터를 LabelEncoder( ) 를 이용하여 수치화
- SVC 모델을 이용하여 벡터화된 데이터와 카테고리 데이터를 이용하여 학습
- new_questions의 카테고리를 확인

In [67]:
from sklearn.preprocessing import LabelEncoder

In [54]:
# 고객 질문 데이터
questions

['지방세를 내려면 어떻게 해야됩니까?',
 '은행을 직접방문해도 됩니까?',
 '버스로 가는 방법도 있습니까?',
 '다른 납부방법도 있습니까?',
 '사이트 주소가 어떻게 됩니까?',
 '다른곳에서는 납부할수 없습니까?',
 '지방세는 조회할 수 있습니까?',
 '어떻게 납부합니까?',
 '은행 어플에서도 됩니까?',
 '서울시주최 페스티벌 예매해놨는데 예정대로 진행됩니까?',
 '코로나로 다른 축제들은 취소됐는데 이건 취소 안됩니까?',
 '환불할수 있습니까?',
 '환불수수료 있습니까?',
 '코로나때문에 못가는건데도 수수료 내야합니까?',
 '참석했다가 코로나 걸리면 보상해줍니까?',
 '그래도 걸리면 보상해줍니까?',
 '확산세가 더 강해지면 취소될 가능성 있습니까?',
 '그럼 전액 환불되는겁니까?',
 '청년저축계좌 지금 신청할 수 있습니까?',
 '그럼 다른 정책은 없습니까?',
 '그건 뭡니까?',
 '서울시민 대상입니까?',
 '얼마를 추가 적립해줍니까?',
 '모집인원은 어떻게 됩니까?',
 '신청기간은 언제까지입니까?',
 '네 주소지가 서울이면 되는겁니까?',
 '그럼 총 어느정도 혜택을 받을 수 있습니까?',
 '3년동안 계속 10만원씩 해야합니까?',
 '일도 쉬면 안됩니까?',
 '또 다른 조건 있습니까?',
 '저희가 작년에 결혼했는데 신혼부부 조건이 맞습니까?',
 '대출 상품이 뭐가 있습니까?',
 '어떤 은행에서 받을 수 있습니까?',
 '버팀목 대출 받으려면 조건이 있습니까?',
 '서울소재 주택만 가능한겁니까?',
 '대출한도는 어떻게 됩니까?',
 '100% 다 대출받을 수 있습니까?',
 '그럼 나머지 10%에 대해서는 그냥 대출을 받을 수 있습니까?',
 '그럼 그 한정된 곳에서 집을 찾아야합니까?',
 '대출 한도는 어떻게 됩니까?',
 '금리는 얼마입니까?',
 '그렇게 낮은편이 아닌데 다른 혜택은 없습니까?',
 '그게 뭡니까?',
 '그럼 1% 이율입니까?',
 '대출기간은 어떻게 됩니까?',
 '5월에 퇴

In [58]:
# (독립 변수)
okt = Okt()

# 토큰화
def tokenize(text):
    # 특수문자 제거 및 정규화 추가
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    return okt.morphs(text)

# 벡터화
tfidf = TfidfVectorizer(
    tokenizer= tokenize,
    ngram_range= (1, 2),
    min_df= 1
)

In [ ]:
# (종속 변수)
# 카테고리 '일반행정 문의', '대중교통 안내' - LabelEncoder() 사용해 수치화
le = LabelEncoder
df_combined['카테고리']

'일반행정 문의'